<a href="https://colab.research.google.com/github/pintophilip/Agentic-AI-Application/blob/main/Multi_Agent_Research_and_Summarization_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langchain langchainhub faiss-cpu llama-index groq
!pip install duckduckgo-search  # for web search agent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [16]:
!pip install --upgrade langgraph langchain langchain_groq

# 🔐 Set up your Groq API Key
import os
os.environ["GROQ_API_KEY"] = "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"

from langchain_groq import ChatGroq
from langchain.tools import Tool
from langgraph.graph import StateGraph, END
from langchain.agents import initialize_agent, AgentType
import os

In [17]:
llm = ChatGroq(
    model="llama3-70b-8192",  # Groq model ID for llama-3.3
    api_key=os.getenv("GROQ_API_KEY")
)

Web Research Tool

In [18]:
from duckduckgo_search import DDGS

def search_web(query):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        return "\n".join([r['body'] for r in results])

In [19]:
web_search_tool = Tool.from_function(
    name="WebSearchTool",
    func=search_web,
    description="Useful for current or latest events."
)

📚 RAG Tool (Vector DB setup using FAISS + sample text data)

In [20]:
!pip install langchain-community
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
# We are importing this specifically to load from texts
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter

# Load & embed data
documents = ["LangGraph is a library for building agent workflows...", "Tool calling allows LLMs to use external tools..."]
# Instead of TextLoader, directly create Document objects
docs = [Document(page_content=text) for text in documents]

# The rest remains as before
splitter = CharacterTextSplitter(chunk_size=300)
docs = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(docs, embeddings)

def rag_search(query):
    result = db.similarity_search(query, k=2)
    return "\n".join([doc.page_content for doc in result])

rag_tool = Tool.from_function(
    name="RAGTool",
    func=rag_search,
    description="Useful for querying internal dataset knowledge."
)

<ipython-input-20-e723c37bba92>:18: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


🧭 Router Agent

In [21]:
def route_query(state):
    query = state["query"].lower()
    if "latest" in query or "current" in query:
        return "web_agent"
    elif "langgraph" in query or "tool" in query or "dataset" in query:  # keywords for internal dataset
        return "rag_agent"
    else:
        return "llm_agent"

📝 Summarization Agent

In [22]:
def summarizer(state):
    context = state["info"]
    summary = llm.invoke(f"Summarize the following:\n\n{context}")
    return {"query": state["query"], "result": summary}

In [23]:
#define state and graph nodes
from typing import TypedDict

class AgentState(TypedDict):
    query: str
    info: str
    result: str

def web_agent(state: AgentState) -> AgentState:
    info = search_web(state["query"])
    return {"query": state["query"], "info": info}

def rag_agent(state: AgentState) -> AgentState:
    info = rag_search(state["query"])
    return {"query": state["query"], "info": info}

def llm_agent(state: AgentState) -> AgentState:
    response = llm.invoke(state["query"])
    return {"query": state["query"], "info": response}

In [24]:
builder = StateGraph(AgentState)

# Add all 4 nodes
builder.add_node("web_agent", web_agent)
builder.add_node("rag_agent", rag_agent)
builder.add_node("llm_agent", llm_agent)
builder.add_node("summarizer", summarizer)

# Router logic
builder.set_conditional_entry_point(route_query)

# Connect to summarizer from each branch
builder.add_edge("web_agent", "summarizer")
builder.add_edge("rag_agent", "summarizer")
builder.add_edge("llm_agent", "summarizer")

# End
builder.add_edge("summarizer", END)

# Compile the final graph
graph = builder.compile()

In [25]:
# LLM route test
res1 = graph.invoke({"query": "Explain the concept of transformers in NLP."})
print("🤖 LLM Response:\n", res1["result"])

# RAG route test
res2 = graph.invoke({"query": "What is LangGraph used for?"})
print("📚 RAG Response:\n", res2["result"])

# Web route test
res3 = graph.invoke({"query": "What is the latest AI trend?"})
print("🌐 Web Response:\n", res3["result"])

🤖 LLM Response:
 content="The text describes the concept of transformers in Natural Language Processing (NLP). Here's a summary:\n\n**Background**: Before transformers, Recurrent Neural Networks (RNNs) were widely used for sequence-to-sequence tasks like machine translation, text summarization, and language modeling. However, RNNs have limitations, including sequential processing and fixed-length context.\n\n**Transformer Architecture**: The transformer architecture, introduced in 2017, addresses these limitations by introducing self-attention mechanisms and parallelization. The key components are:\n\n* Self-Attention: allows the model to attend to all elements in the input sequence simultaneously\n* Multi-Head Attention: uses multiple attention mechanisms in parallel to capture multiple aspects of the input sequence\n* Encoder-Decoder Structure: consists of an encoder that takes in a sequence of tokens and outputs a continuous representation, and a decoder that generates the output se